In [ ]:
### Required Libraries ###
from datetime import datetime
from dateutil.relativedelta import relativedelta


### Functionality Helper Functions ###
def parse_int(n):
    """
    Securely converts a non-integer value to integer.
    """
    try:
        return int(n)
    except ValueError:
        return float("nan")


def parse_float(n):
    """
    Securely converts a non-numeric value to float.
    """
    try:
        return float(n)
    except ValueError:
        return float("nan")
        
        
def build_validation_result(is_valid, violated_slot, message_content):
    """
    Define a result message structured as Lex response.
    """
    if message_content is None:
        return {"isValid": is_valid, "violatedSlot": violated_slot}

    return {
        "isValid": is_valid,
        "violatedSlot": violated_slot,
        "message": {"contentType": "PlainText", "content": message_content},
    }


### Dialog Actions Helper Functions ###
def get_slots(intent_request):
    """
    Fetch all the slots and their values from the current intent.
    """
    return intent_request["currentIntent"]["slots"]


def elicit_slot(session_attributes, intent_name, slots, slot_to_elicit, message):
    """
    Defines an elicit slot type response.
    """

    return {
        "sessionAttributes": session_attributes,
        "dialogAction": {
            "type": "ElicitSlot",
            "intentName": intent_name,
            "slots": slots,
            "slotToElicit": slot_to_elicit,
            "message": message,
        },
    }


def delegate(session_attributes, slots):
    """
    Defines a delegate slot type response.
    """

    return {
        "sessionAttributes": session_attributes,
        "dialogAction": {"type": "Delegate", "slots": slots},
    }


def close(session_attributes, fulfillment_state, message):
    """
    Defines a close slot type response.
    """

    response = {
        "sessionAttributes": session_attributes,
        "dialogAction": {
            "type": "Close",
            "fulfillmentState": fulfillment_state,
            "message": message,
        },
    }

    return response


def validate_data(first_name, last_name, last_four, amount):

    # The 'firstName' should only contain letters and no numbers or special characters. 
    if first_name is not None:
        if not first_name.isalpha():
            return build_validation_result(False, 'firstName', "Please type your first name again. This should only contain letters and no numbers and/or special characters.")
        
    # The 'lastName' should only contain letters and no numbers or special characters. 
    if last_name is not None:
        if not last_name.isalpha():
            return build_validation_result(False, 'lastName', "Please type your last name again. This should only contain letters and no numbers and/or special characters.")   
    
    # The `lastFour` should only contain numbers and no letters or special characters.  
    if last_four is not None:
        if not last_four.isnumeric():
            return build_validation_result(False, 'lastFour', "Please type the last four digits of your credit card again. This should only consists of numbers and no letters and/or special characters.")    
      
    # The `amount` should only contain numbers and no letters or special characters.  
    if amount is not None:
        if not amount.isnumeric():
            return build_validation_result(False, 'amount', "Please type the dollar amount again. This should only consists of numbers and no letters and/or special characters.")    

    return build_validation_result(True, None, None)

### Intents Handlers ###
def file_fraud(intent_request):
    """
    Performs dialog management and fulfillment for filing a fraud claim.
    """

    input_first_name = get_slots(intent_request)["firstName"]
    input_last_name = get_slots(intent_request)["lastName"]
    input_last_four = get_slots(intent_request)["lastFour"]
    input_amount = get_slots(intent_request)["amount"]

    source = intent_request["invocationSource"]
    
    if source == 'DialogCodeHook':
        slots = get_slots(intent_request)
        validation_result = validate_data(input_first_name, input_last_name, input_last_four, input_amount)
        
        if not validation_result['isValid']:
            slots[validation_result['violatedSlot']] = None
            return elicit_slot(intent_request['sessionAttributes'],
                intent_request['currentIntent']['name'],
                slots, 
                validation_result['violatedSlot'],
                validation_result['message'])
        output_session_attributes = intent_request['sessionAttributes'] if intent_request['sessionAttributes'] is not None else {}
        return delegate(output_session_attributes, get_slots(intent_request))
  
    fraud_result = fraud_action(input_card_possession)
    return close(intent_request['sessionAttributes'],
                 'Fulfilled',
                 {'contentType': 'PlainText',
                  'content': 'We will file a fraud report, cancel your card and mail you a new one.'})

### Intents Dispatcher ###
def dispatch(intent_request):
    """
    Called when the user specifies an intent for this bot.
    """

    intent_name = intent_request["currentIntent"]["name"]

    # Dispatch to bot's intent handlers
    if intent_name == "identifyFraud":
        return file_fraud(intent_request)

    raise Exception("Intent with name " + intent_name + " not supported")


### Main Handler ###
def lambda_handler(event, context):
    """
    Route the incoming request based on intent.
    The JSON body of the request is provided in the event slot.
    """
    
    return dispatch(event)
